<a href="https://colab.research.google.com/github/SanaImran0/fine-tuning/blob/main/Finetuning_MedBot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.4/485.4 kB 14.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 15.4 MB/s eta 0:00:00


In [ ]:
!pip install bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 48.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 47.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 34.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 39.0 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from peft import LoraConfig, prepare_model_for_kbit_training, get_peft_model
from transformers import TrainingArguments, Trainer, DataCollatorForLanguageModeling
from datasets import load_dataset
from sklearn.model_selection import train_test_split
import torch
import os
import wandb

In [ ]:
import torch.utils.checkpoint
torch.utils.checkpoint.checkpoint = lambda function, *args, **kwargs: function(*args, **kwargs)

In [ ]:
model_name = "stanford-crfm/BioMedLM"
dataset_name = "FreedomIntelligence/medical-o1-reasoning-SFT"
output_dir = "./medical-o1-reasoning-SFT"
os.makedirs(output_dir, exist_ok=True)

In [ ]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16
)

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map="auto",
)

In [ ]:
dataset = load_dataset(dataset_name, 'en')
print(f"Dataset structure: {dataset}")
print(f"Dataset columns: {dataset['train'].column_names}")
print(f"Sample data: {dataset['train'][0]}")

Dataset structure: DatasetDict({
    train: Dataset({
        features: ['Question', 'Complex_CoT', 'Response'],
        num_rows: 25371
    })
})
Dataset columns: ['Question', 'Complex_CoT', 'Response']
Sample data: {'Question': 'A 61-year-old woman with a long history of involuntary urine loss during activities like coughing or sneezing but no leakage at night undergoes a gynecological exam and Q-tip test. Based on these findings, what would cystometry most likely reveal about her residual volume and detrusor contractions?', 'Complex_CoT': "Okay, let's think about this step by step. There's a 61-year-old woman here who's been dealing with involuntary urine leakages whenever she's doing something that ups her abdominal pressure like coughing or sneezing. This sounds a lot like stress urinary incontinence to me. Now, it's interesting that she doesn't have any issues at night; she isn't experiencing leakage while sleeping. This likely means her bladder's ability to hold urine is fine wh

In [ ]:
def preprocess_function(examples):
    formatted_inputs = []
    for question, cot, response in zip(examples["Question"], examples["Complex_CoT"], examples["Response"]):
        formatted_text = f"Question: {question}\nReasoning: {cot}\nAnswer: {response}"
        formatted_inputs.append(formatted_text)

    tokenized = tokenizer(
        formatted_inputs,
        truncation=True,
        padding="max_length",
        max_length=256,
        return_tensors="pt"
    )
    tokenized["labels"] = tokenized["input_ids"].clone()

    return tokenized

In [ ]:
tokenized_dataset = dataset.map(
    preprocess_function,
    batched=True,
    remove_columns=["Question", "Complex_CoT", "Response"]
)

Map:   0%|          | 0/25371 [00:00<?, ? examples/s]

In [ ]:
model = prepare_model_for_kbit_training(model)

In [ ]:
def print_trainable_modules(model):
    modules = []
    for name, module in model.named_modules():
        if isinstance(module, torch.nn.Linear):
            modules.append(name)

    print("Potential LoRA target modules:")
    for module in modules:
        print(f"  - {module}")

    patterns = ["query", "key", "value", "attention", "mlp", "dense"]
    print("\nModules containing common patterns:")
    for pattern in patterns:
        matching = [m for m in [] if pattern in m.lower()]
        if matching:
            print(f"Pattern '{pattern}':")
            for m in matching[:5]:
                print(f"  - {m}")
            if len(matching) > 5:
                print(f"  ... and {len(matching) - 5} more")
    return modules

trainable_modules = print_trainable_modules(model)

Potential LoRA target modules:
  - transformer.h.0.attn.c_attn
  - transformer.h.0.attn.c_proj
  - transformer.h.0.mlp.c_fc
  - transformer.h.0.mlp.c_proj
  - transformer.h.1.attn.c_attn
  - transformer.h.1.attn.c_proj
  - transformer.h.1.mlp.c_fc
  - transformer.h.1.mlp.c_proj
  - transformer.h.2.attn.c_attn
  - transformer.h.2.attn.c_proj
  - transformer.h.2.mlp.c_fc
  - transformer.h.2.mlp.c_proj
  - transformer.h.3.attn.c_attn
  - transformer.h.3.attn.c_proj
  - transformer.h.3.mlp.c_fc
  - transformer.h.3.mlp.c_proj
  - transformer.h.4.attn.c_attn
  - transformer.h.4.attn.c_proj
  - transformer.h.4.mlp.c_fc
  - transformer.h.4.mlp.c_proj
  - transformer.h.5.attn.c_attn
  - transformer.h.5.attn.c_proj
  - transformer.h.5.mlp.c_fc
  - transformer.h.5.mlp.c_proj
  - transformer.h.6.attn.c_attn
  - transformer.h.6.attn.c_proj
  - transformer.h.6.mlp.c_fc
  - transformer.h.6.mlp.c_proj
  - transformer.h.7.attn.c_attn
  - transformer.h.7.attn.c_proj
  - transformer.h.7.mlp.c_fc
  - tran

In [ ]:
lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=trainable_modules,
)

In [ ]:
model = get_peft_model(model, lora_config)

/usr/local/lib/python3.11/dist-packages/peft/tuners/tuners_utils.py:543: UserWarning: Model with `tie_word_embeddings=True` and the tied_target_modules=['lm_head'] are part of the adapter. This can lead to complications, for example when merging the adapter or converting your model to formats other than safetensors. See for example https://github.com/huggingface/peft/issues/2018.
  warnings.warn(


In [ ]:
print("Trainable parameters:")
trainable_params = 0
all_params = 0
for name, param in model.named_parameters():
    all_params += param.numel()
    if param.requires_grad:
        trainable_params += param.numel()
        print(f"{name}: {param.shape}")
print(f"Trainable parameters: {trainable_params:,} ({100 * trainable_params / all_params:.2f}% of {all_params:,})")

Trainable parameters:
base_model.model.transformer.h.0.attn.c_attn.lora_A.default.weight: torch.Size([16, 2560])
base_model.model.transformer.h.0.attn.c_attn.lora_B.default.weight: torch.Size([7680, 16])
base_model.model.transformer.h.0.attn.c_proj.lora_A.default.weight: torch.Size([16, 2560])
base_model.model.transformer.h.0.attn.c_proj.lora_B.default.weight: torch.Size([2560, 16])
base_model.model.transformer.h.0.mlp.c_fc.lora_A.default.weight: torch.Size([16, 2560])
base_model.model.transformer.h.0.mlp.c_fc.lora_B.default.weight: torch.Size([10240, 16])
base_model.model.transformer.h.0.mlp.c_proj.lora_A.default.weight: torch.Size([16, 10240])
base_model.model.transformer.h.0.mlp.c_proj.lora_B.default.weight: torch.Size([2560, 16])
base_model.model.transformer.h.1.attn.c_attn.lora_A.default.weight: torch.Size([16, 2560])
base_model.model.transformer.h.1.attn.c_attn.lora_B.default.weight: torch.Size([7680, 16])
base_model.model.transformer.h.1.attn.c_proj.lora_A.default.weight: torch.

In [ ]:
training_args = TrainingArguments(
    output_dir=output_dir,
    learning_rate=2e-4,
    per_device_train_batch_size=1,
    gradient_accumulation_steps=16,
    num_train_epochs=3,
    weight_decay=0.01,
    warmup_ratio=0.1,
    evaluation_strategy="no",
    save_strategy="steps",
    save_steps=500,
    fp16=True,
    torch_compile=False,
    dataloader_num_workers=0,
    optim="adamw_8bit",
)

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
if "validation" not in dataset:
    train_valid_split = tokenized_dataset["train"].train_test_split(test_size=0.1, seed=42)
    train_dataset = train_valid_split["train"]
    eval_dataset = train_valid_split["test"]
else:
    train_dataset = tokenized_dataset["train"]
    eval_dataset = tokenized_dataset["validation"]

In [ ]:
train_dataset = train_dataset.select(range(min(1000, len(train_dataset))))
eval_dataset = eval_dataset.select(range(min(100, len(eval_dataset))))

In [ ]:
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False
)

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    data_collator=data_collator,
)
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: sanaimran6722 (sanaimran6722-octaloop-technologies) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:632: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.


Step,Training Loss


/usr/local/lib/python3.11/dist-packages/peft/utils/save_and_load.py:230: UserWarning: Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.
  warnings.warn("Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.")


TrainOutput(global_step=186, training_loss=3.0839216991137435, metrics={'train_runtime': 2481.1947, 'train_samples_per_second': 1.209, 'train_steps_per_second': 0.075, 'total_flos': 1.154429459890176e+16, 'train_loss': 3.0839216991137435, 'epoch': 2.96})

In [ ]:
trainer.save_model(output_dir + "/final_model")
model.save_pretrained(output_dir + "/peft_adapter")
tokenizer.save_pretrained(output_dir + "/peft_adapter")

/usr/local/lib/python3.11/dist-packages/peft/utils/save_and_load.py:230: UserWarning: Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.
  warnings.warn("Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.")


('./medical-o1-reasoning-SFT/peft_adapter/tokenizer_config.json',
 './medical-o1-reasoning-SFT/peft_adapter/special_tokens_map.json',
 './medical-o1-reasoning-SFT/peft_adapter/vocab.json',
 './medical-o1-reasoning-SFT/peft_adapter/merges.txt',
 './medical-o1-reasoning-SFT/peft_adapter/added_tokens.json',
 './medical-o1-reasoning-SFT/peft_adapter/tokenizer.json')

In [ ]:
def generate_medical_response(question, max_length=512):
    prompt = f"Question: {question}\n\nProvide a comprehensive medical answer with potential complications:"

    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

    with torch.no_grad():
        outputs = model.generate(
            input_ids=inputs["input_ids"],
            attention_mask=inputs["attention_mask"],
            max_length=max_length,
            temperature=0.3,
            top_p=0.9,
            do_sample=True,
            repetition_penalty=1.2,
            no_repeat_ngram_size=3,
            num_return_sequences=1,
        )

    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    response = response.replace(prompt, "").strip()
    return response

In [ ]:
test_question = "What are the potential complications of untreated hypertension?"
generated_response = generate_medical_response(test_question)
print(f"Question: {test_question}")
print(f"Generated response: {generated_response}")

Setting `pad_token_id` to `eos_token_id`:28895 for open-end generation.
/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:632: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.11/dist-packages/torch/utils/checkpoint.py:87: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(


In [ ]:
def evaluate_on_test_set(test_dataset, num_samples=10):
    model.eval()
    results = []

    for i in range(min(num_samples, len(test_dataset))):
        sample = test_dataset[i]
        question = sample["Question"]
        actual_response = sample["Response"]

        generated = generate_medical_response(question)

        results.append({
            "question": question,
            "actual": actual_response,
            "generated": generated
        })

    return results

In [ ]:
if "test" in dataset:
    test_results = evaluate_on_test_set(dataset["test"])
    for i, result in enumerate(test_results):
        print(f"Example {i+1}:")
        print(f"Question: {result['question']}")
        print(f"Actual: {result['actual']}")
        print(f"Generated: {result['generated']}")
        print("-" * 80)